In [ ]:
%reload_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import json
import argparse
import os
from glob import glob
import numpy as np
import torch
from evaluator import evaluator_brats
from networks.mmt import MMT as generator
from torchvision.utils import save_image
from tqdm import tqdm
from configs.config import get_config
from datasets.dataset_brats import BRATS_dataset, RandomGeneratorBRATS
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.nn import Upsample

import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

split = 'val'
batch_size = 1
data_dir = '/home/srivathsa/projects/studies/gad/stanford/preprocess/data_mmt/zero_gad'
ckpt_base = '/home/srivathsa/projects/SubtleGad/train/prototypes/mmt/MMTUNetHybrid/model/'
ckpt_base += 'MMT_random_sford_fb_epo100_bs24_lrg0.0005_8.0_20.0_0.0_0.0_vgg12.0'

db = BRATS_dataset(base_dir=data_dir, split=split,
                               transform=transforms.Compose(
                                   [RandomGeneratorBRATS(flip=False, scale=None)]))
dataloader = DataLoader(db, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)
G = generator(img_size=[160, 192],
                                patch_size=4,
                                in_chans=1,
                                out_chans = 1,
                                embed_dim=96,
                                depths=[2,2,2,2],
                                num_heads=[3,6,12,24],
                                window_size=[5,6],
                                mlp_ratio=4.0,
                                qkv_bias=True,
                                qk_scale=None,
                                drop_rate=0,
                                drop_path_rate=0,
                                ape=False,
                                patch_norm=True,
                                use_checkpoint=False).cuda()

In [ ]:
args = argparse.Namespace()
args.batch_size = 1
args.n_gpu = 1
args.root_path = data_dir
args.seed = 1234

metric_list = []

ckp_files = [f for f in glob('{}/epoch*.pth'.format(ckpt_base))]
ckp_files = sorted(ckp_files, key=lambda f: int(f.split('/')[-1].split('.')[0].split('_')[1]))

for ckpt_path in ckp_files:
#     print('**** EVALUATING {} ****'.format(ckpt_path))
    
    state_dict = torch.load(ckpt_path, map_location='cuda:0')
    G.load_state_dict(state_dict['G'])
    G.eval()

    metrics, metrics_seg, _ = evaluator_brats(args, G, [[0, 2, 3], [0, 1, 2]], split)
    print('metrics?', metrics)
    met_dict = metrics[0][0]
    met_dict['epoch'] = ckpt_path.split('/')[-1]
    metric_list.append(met_dict)
    
    break

In [ ]:
metrics

In [ ]:
metric_list

In [ ]:
with open('{}/metrics_list.json'.format(ckpt_base), 'w') as f:
    f.write(json.dumps(metric_list))

In [1]:
import pandas as pd

fpath = '/home/srivathsa/projects/SubtleGad/train/prototypes/mmt/MMTUNetHybrid/model/\
MMT_single_no_gan_epo100_bs24_lrg0.0005_5.0_20.0_0.0_0.0_vgg0/metrics_list.json'

df_metrics = pd.read_json(fpath)
df_metrics = df_metrics.groupby('epoch').mean()

In [2]:
df_metrics

,mse,mae,ssim,psnr
epoch,,,,
epoch_0.pth,0.007251,0.041335,0.792181,22.170261
epoch_1.pth,0.005357,0.033720,0.847446,23.330373
epoch_10.pth,0.002560,0.022617,0.914762,26.831447
epoch_11.pth,0.002587,0.022723,0.915310,26.841024
epoch_12.pth,0.002522,0.022542,0.915895,26.857292
...,...,...,...,...
epoch_95.pth,0.002165,0.020746,0.927174,27.816346
epoch_96.pth,0.002166,0.020750,0.927184,27.816339
epoch_97.pth,0.002166,0.020749,0.927098,27.813717


In [3]:
df_metrics[df_metrics.mse == df_metrics.mse.min()]

,mse,mae,ssim,psnr
epoch,,,,
epoch_72.pth,0.002155,0.020747,0.926992,27.809976


In [4]:
df_metrics[df_metrics.mae == df_metrics.mae.min()]

,mse,mae,ssim,psnr
epoch,,,,
epoch_89.pth,0.002166,0.020737,0.927106,27.810899


In [5]:
df_metrics[df_metrics.ssim == df_metrics.ssim.max()]

,mse,mae,ssim,psnr
epoch,,,,
epoch_96.pth,0.002166,0.02075,0.927184,27.816339


In [6]:
df_metrics[df_metrics.psnr == df_metrics.psnr.max()]

,mse,mae,ssim,psnr
epoch,,,,
epoch_87.pth,0.002161,0.020737,0.927057,27.819317


In [ ]:
df_metrics[df_metrics.index == 'epoch_56.pth']

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.set_cmap('gray')

data = np.load('/home/srivathsa/test_vol.npy')
print(data.shape)

In [ ]:
plt.imshow(data[89])